<a href="https://colab.research.google.com/github/JebaMercy14/Rare-Word-Text-summarization/blob/main/Rare_Word_Text_summarization_Bert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center> Rare Word - Text summarization </center>

# Prerequisite Downloads

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00


In [ ]:
!pip install -U nltk # To Upgrade NLTK to >3.5 for METEOR score

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle #directory

In [ ]:
!cp kaggle.json ~/.kaggle/ #kaggle api

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail #downloading dataset to be used

100% 503M/503M [00:25<00:00, 22.4MB/s]
100% 503M/503M [00:25<00:00, 20.5MB/s]


In [ ]:
! unzip newspaper-text-summarization-cnn-dailymail.zip #unzipping

Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [ ]:
import nltk

print('The nltk version is {}.'.format(nltk.__version__))

The nltk version is 3.8.1.


# Import Libraries

In [ ]:
!pip install rouge --quiet

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import BertForSequenceClassitication, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from rouge import Rouge

# Import Datasets

In [ ]:

class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=512, max_output_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        article = self.data.iloc[index]["article"]
        summary = self.data.iloc[index]["highlights"]
        input_ids = self.tokenizer.encode(article, max_length=self.max_input_length, truncation=True, padding="max_length")
        output_ids = self.tokenizer.encode(summary, max_length=self.max_output_length, truncation=True, padding="max_length")
        return {"input_ids": input_ids, "attention_mask": [int(token_id != 0) for token_id in input_ids], "decoder_input_ids": output_ids[:-1], "decoder_attention_mask": [1] * (len(output_ids) - 1), "labels": output_ids[1:]}

In [ ]:

train_df = pd.read_csv("cnn_dailymail/train.csv")
test_df = pd.read_csv("cnn_dailymail/test.csv")
val_df = pd.read_csv("cnn_dailymail/validation.csv")

In [ ]:

from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# Load a smaller pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

# Load a smaller pre-trained model
model = BertForSequenceClassitication.from_pretrained("google/pegasus-cnn_dailymail")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
#model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")

# Data Cleaning

In [ ]:
train_dataset = SummarizationDataset(train_df, tokenizer)
val_dataset = SummarizationDataset(val_df, tokenizer)

In [ ]:

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    decoder_input_ids = [item["decoder_input_ids"] for item in batch]
    decoder_attention_mask = [item["decoder_attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]
    max_input_length = max(len(ids) for ids in input_ids)
    max_output_length = max(len(ids) for ids in decoder_input_ids)
    input_ids = [ids + [0] * (max_input_length - len(ids)) for ids in input_ids]
    attention_mask = [mask + [0] * (max_input_length - len(mask)) for mask in attention_mask]
    decoder_input_ids = [ids + [0] * (max_output_length - len(ids)) for ids in decoder_input_ids]
    decoder_attention_mask = [mask + [0] * (max_output_length - len(mask)) for mask in decoder_attention_mask]
    labels = [ids + [-100] * (max_output_length - len(ids)) for ids in labels]
    return {"input_ids": torch.tensor(input_ids), "attention_mask": torch.tensor(attention_mask), "decoder_input_ids": torch.tensor(decoder_input_ids), "decoder_attention_mask": torch.tensor(decoder_attention_mask), "labels": torch.tensor(labels)}

# Test - Train Split

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=2,collate_fn=collate_fn)

In [ ]:

optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=10000)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
len(train_loader),len(val_loader)

(143557, 6684)

# Modeling

In [ ]:

from tqdm import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(10):
    train_loss = 0
    for step, batch in tqdm(enumerate(train_loader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        loss = model(**batch).loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        train_loss += loss.item()

        if step % 100 == 0 and step > 0:
            print("Step-{},Train Loss-{}".format(step, loss.item()))

    train_loss /= len(train_loader)

    val_loss = 0
    model.eval()
    with torch.no_grad():
        for step, batch in tqdm(enumerate(val_loader)):
            batch = {k: v.to(device) for k, v in batch.items()}
            loss = model(**batch).loss
            val_loss += loss.item()

            if step % 100 == 0 and step > 0:
                print("Step-{},Val Loss-{}".format(step, loss.item()))

    val_loss /= len(val_loader)

    model.train()

    print(f"Epoch {epoch+1} train loss: {train_loss:.4f} val loss: {val_loss:.4f}")

101it [01:27,  1.13it/s]

Step-100,Train Loss-9.630538940429688


201it [02:54,  1.15it/s]

Step-200,Train Loss-9.436802864074707


301it [04:21,  1.14it/s]

...

Step-900,Train Loss-1.3972314596176147


1001it [14:34,  1.15it/s]

Step-1000,Train Loss-0.32566529512405396


1076it [15:40,  1.14it/s]


KeyboardInterrupt: ignored

In [ ]:
model.save_pretrained("fine_tuned_pegasus")
tokenizer.save_pretrained("fine_tuned_pegasus")

('fine_tuned_pegasus/tokenizer_config.json',
 'fine_tuned_pegasus/special_tokens_map.json',
 'fine_tuned_pegasus/spiece.model',
 'fine_tuned_pegasus/added_tokens.json')

In [ ]:

model = PegasusForConditionalGeneration.from_pretrained("fine_tuned_pegasus")
tokenizer = PegasusTokenizer.from_pretrained("fine_tuned_pegasus")

test_dataset = SummarizationDataset(test_df, tokenizer)

test_loader = DataLoader(test_dataset, batch_size=1,collate_fn=collate_fn)

len(test_loader)

11490

# Prediction

In [ ]:

model.to(device)
model.eval()
predictions = []
with torch.no_grad():
    for step, batch in tqdm(enumerate(test_loader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        output_ids = model.generate(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], max_length=128, decoder_start_token_id=tokenizer.pad_token_id)
        batch_predictions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]
        predictions.extend(batch_predictions)
        if step==100:
            break # breaking after generating 100 predictions.. since it's going to take long to predict on entire set

100it [05:20,  3.20s/it]


In [ ]:
len(predictions)

101

In [ ]:

# Save the predictions to a CSV file
test_df = test_df[:101]# for 100 predicitons only
print(len(test_df))
test_df["predictions"] = predictions
test_df.to_csv("test_predictions.csv", index=False)

101


<ipython-input-28-2ab9a0010fec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["predictions"] = predictions


In [ ]:
test_df

,id,article,highlights,predictions
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,'''''''''''''''''''''In a world where animals ...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,Kumar Kumar Kumar Kumar Kumar Kumar Kumar Kuma...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,manager manager manager manager manager manage...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Net Net Net Net Net Net Net Net Net Net Net Ne...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",Jenner Jenner Jenner Jenner Jenner Jenner Jenn...
...,...,...,...,...
96,5cf4682cd03238d5867027ce9492b626cd1ed011,"Jameela spent £3,000 on having all her amalgam...","Jameela Jamil, 29, is convinced dental work tr...",Jamil Jamil Jamil Jamil Jamil Jamil Jamil Jami...
97,3815d19af18ff22be6ad6095722d7367bb7271af,A paramedic who pretended he was gay to get cl...,"Christopher Bridger, 25, attacked three women ...",Bridge Bridge Bridge Bridge Bridge Bridge Brid...
98,fb207604ffa7e8371c622840445825db8993d4d2,"Paris Saint-Germain face Nice on Saturday, hop...",Paris Saint-Germain captain Thiago Silva suffe...,Silva Silva Silva Silva Silva Silva Silva Silv...
99,d25d52c434a13c1df5faa593e8a097d2f501a2b6,"(CNN)You know the phrase ""dodging a bullet""? F...",.50-caliber bullets equipped with optical sens...,bullet bullet bullet bullet bullet bullet bull...


# Calculating scores

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Calculate the ROUGE scores between the predicted summaries and the actual summaries
rouge = Rouge()
scores = rouge.get_scores(predictions, test_df["highlights"].tolist(), avg=True)

# Print the ROUGE scores
print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")

ROUGE-1: 0.3234
ROUGE-2: 0.1243
ROUGE-L: 0.3057


# Rare Word Analysis

In [ ]:


def rare_word_analysis(data_column):
    vocabulary = set()
    word_count = {}

    # Calculate word frequencies
    for text in data_column:
        words = text.split()
        for word in words:
            vocabulary.add(word)
            word_count[word] = word_count.get(word, 0) + 1

    # Calculate total_count and count
    total_count = len(vocabulary)
    threshold = 10  # You can set the threshold as per your requirement
    count = sum(1 for count in word_count.values() if count < threshold)

    return total_count, count

# Perform rare word analysis for the 'text' column in 'test_df'
text_data_column = test_df['predictions']
text_total_count, text_count = rare_word_analysis(text_data_column)

# Calculate the top most common words in 'text'
text_top_common = text_total_count - text_count

print("Rare Word Analysis for 'text' column:")
print("Total unique words:", text_total_count)
print("Number of rare words:", text_count)
print("Number of top most common words:", text_top_common)

Rare Word Analysis for 'text' column:
Total unique words: 2029
Number of rare words: 1869
Number of top most common words: 160
